In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import glob

In [2]:
!git clone "https://github.com/AlexeyAB/darknet.git" "/content/drive/MyDrive/Helmet_Detection/Helmet/darknet"

Cloning into '/content/drive/MyDrive/Helmet_Detection/Helmet/darknet'...
remote: Enumerating objects: 15313, done.
remote: Total 15313 (delta 0), reused 0 (delta 0), pack-reused 15313
Receiving objects: 100% (15313/15313), 13.71 MiB | 5.77 MiB/s, done.
Resolving deltas: 100% (10405/10405), done.
Checking out files: 100% (2044/2044), done.


In [3]:
%cd "/content/drive/MyDrive/Helmet_Detection/Helmet/darknet"
!make

/content/drive/MyDrive/Helmet_Detection/Helmet/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ [-Wunused-variable]
                 int n = _write(client, outputbuf, outlen);
                     ^
./src/http_stream.cpp: In function ‘void set_track_id(detection*, int, float, float, float, int, int, int)’:
./src/http_stream.cpp:867:27: warning: comparison between signed and unsigned integer expressions [-Wsign-compare]
         for (int i = 0; i < v.size(); ++i) {
                       

In [4]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [5]:
!wget "https://pjreddie.com/media/files/darknet53.conv.74"

--2021-10-07 16:55:16--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  21.7MB/s    in 8.4s    

2021-10-07 16:55:25 (18.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [6]:
!sed -i "s/OPENCV=0/OPENCV=1/" Makefile
!sed -i "s/GPU=0/GPU=1/" Makefile
!sed -i "s/CUDNN=0/CUDNN=1/" Makefile
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1150:10: note: ...thi

In [7]:
!mkdir "data/obj/"

In [8]:
%cd ..

/content/drive/MyDrive/Helmet_Detection/Helmet


In [13]:
img_list = glob.glob("darknet/data/obj/*.png")
train_index = int(len(img_list)*0.8)
train_file = open("darknet/data/train.txt","w")
train_file.write("\n".join(img_list[:train_index]))
train_file.close()
test_file = open("darknet/data/test.txt","w")
test_file.write("\n".join(img_list[train_index:]))
test_file.close()

In [15]:
!echo "Helmet" > darknet/data/obj.names
!echo -e 'classes = 1\ntrain  = darknet/data/train.txt\nvalid  = darknet/data/test.txt\nnames = darknet/data/obj.names\nbackup = /content/drive/MyDrive/Helmet_Detection/Helmet/darknet/backup/' > darknet/data/obj.data

In [16]:
!darknet/darknet detector train darknet/data/obj.data darknet/cfg/yolov3_training.cfg darknet/darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.645470), count: 4, class_loss = 2.026437, iou_loss = 3.930744, total_loss = 5.957181 
 total_bbox = 219219, rewritten_bbox = 0.090321 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.831173), count: 1, class_loss = 0.152231, iou_loss = 0.525281, total_loss = 0.677512 
 total_bbox = 219220, rewritten_bbox = 0.090320 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (m